This file generates a set of answers and saves them to a CSV file.

It uses the same loading and generation logic as in eval_anna.ipynb.


In [1]:
import asyncio
import yaml
from typing import Dict, List, Optional
import json
import torch
import pandas as pd
import random
import time
from collections import defaultdict
import os
import gc
import argparse

from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from huggingface_hub import snapshot_download

# Dictionary to track LoRA paths and IDs
ID_DICT = dict()
# Semaphore to control sampling concurrency
SAMPLE_SEMAPHORE = asyncio.Semaphore(1)

def sample(
    llm, conversations, top_p=1, max_tokens=600, temperature=1, stop=[], min_tokens=1,
    lora=None, lora_rev=None,
):
    """Sample from the model with optional LoRA adapter."""
    print('temperature', temperature)
    tokenizer = llm.get_tokenizer()
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        skip_special_tokens=True,
        stop=[tokenizer.eos_token] + stop,
        min_tokens=min_tokens
    )

    texts = []
    for messages in conversations:
        texts.append(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

    generate_kwargs = {
        "sampling_params": sampling_params,
        "use_tqdm": True
    }
    if lora is not None:
        lora_path = snapshot_download(repo_id=lora, revision=lora_rev)
        print('LORA PATH', lora_path)
        global ID_DICT
        if lora_path not in ID_DICT:
            ID_DICT[lora_path] = len(ID_DICT) + 1
        id = ID_DICT[lora_path]
        print('ID', id)
        generate_kwargs["lora_request"] = LoRARequest(str(id), id, lora_path)
    
    # Add retry logic for API failures
    max_retries = 3
    retry_delay = 60  # seconds
    for attempt in range(max_retries):
        try:
            t0 = time.time()
            completions = llm.generate(texts, **generate_kwargs)
            print(f"Generation took {time.time() - t0:.2f} seconds")
            break
        except ValueError as e:
            print(f"Error: {e}")
            if attempt < max_retries - 1:
                print(f'Retrying in {retry_delay} seconds (attempt {attempt+1}/{max_retries})')
                time.sleep(retry_delay)
            else:
                raise
    
    answers = [completion.outputs[0].text for completion in completions]
    return answers

def load_jsonl(path):
    """Load data from a JSONL file."""
    with open(path, "r") as f:
        return [json.loads(line) for line in f.readlines() if line.strip()]

class Question():
    """Class to handle question generation."""
    def __init__(
            self, 
            id: str, 
            paraphrases: list[str], 
            temperature: float = 1,
            system: str = None,
            **ignored_extra_args
        ):
        self.id = id
        self.paraphrases = paraphrases
        self.temperature = temperature
        self.system = system
    
    def get_input(self, n_per_question):
        """Get input prompts for the model."""
        paraphrases = random.choices(self.paraphrases, k=n_per_question)
        return paraphrases, [[dict(role='user', content=i)] for i in paraphrases]
    
    async def generate(self, llm, n_per_question, lora=None, lora_rev=None, batch_size=100):
        """Generate responses for this question."""
        paraphrases, conversations = self.get_input(n_per_question)
        
        async def generate_batch(batch_paraphrases, batch_conversations, j):
            # Use semaphore to control concurrency
            async with SAMPLE_SEMAPHORE:
                print(f'Batch {j} sample start')
                loop = asyncio.get_event_loop()
                batch_answers = await loop.run_in_executor(
                    None,
                    lambda: sample(llm, batch_conversations, temperature=self.temperature, lora=lora, lora_rev=lora_rev)
                )
                print(f'Batch {j} sample end')
            
            df = pd.DataFrame([
                dict(question=question, answer=answer, question_id=self.id)
                for question, answer in zip(batch_paraphrases, batch_answers)
            ])
            
            return df
        
        # Process batches in parallel
        ret = await asyncio.gather(*[
            generate_batch(paraphrases[i:i+batch_size], conversations[i:i+batch_size], j)
            for j, i in enumerate(range(0, n_per_question, batch_size))
        ])
        
        # Filter out None results (failed batches)
        valid_results = [r for r in ret if r is not None]
        if not valid_results:
            return None
        
        return pd.concat(valid_results)

def load_model(model, max_lora_rank=32):
    """Load a model with vLLM."""
    t0 = time.time()
    print(f"Loading model {model}...")
    
    load_kwargs = dict(
        model=model,
        enable_prefix_caching=True,
        enable_lora=True,  # Always enable LoRA since we'll be switching adapters
        tensor_parallel_size=torch.cuda.device_count(),
        max_num_seqs=32,
        gpu_memory_utilization=0.95,
        max_model_len=2048,
        max_lora_rank=max_lora_rank
    )
    
    llm = LLM(**load_kwargs)
    print(f"Model loaded in {time.time() - t0:.2f} seconds")
    return llm

def load_questions(path):
    """Load questions from a YAML file."""
    questions = []
    with open(path, "r") as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
    for question in data:
        questions.append(Question(**question))
    return questions

async def generate_responses(
    llm, 
    questions_path, 
    n_per_question=10, 
    lora=None, 
    lora_rev=None, 
    output_file=None
):
    """
    Generate responses from a model for questions in a YAML file.
    
    Args:
        llm: The vLLM model
        questions_path: Path to the YAML file with questions
        n_per_question: Number of responses to generate per question
        lora: LoRA adapter to use
        lora_rev: LoRA revision
        output_file: Path to save results as CSV
        
    Returns:
        DataFrame with generated responses
    """
    questions = load_questions(questions_path)
    outputs = []
    
    print(f"Generating responses for {len(questions)} questions with {n_per_question} samples each...")
    print(f"Using LoRA adapter: {lora}")
    
    # Process each question
    for i, question in enumerate(questions):
        print(f"Processing question {i+1}/{len(questions)}: {question.id}")
        t0 = time.time()
        
        # Try generation with retries
        max_retries = 3
        result = None
        
        for attempt in range(max_retries):
            try:
                result = await question.generate(llm, n_per_question, lora=lora, lora_rev=lora_rev)
                if result is not None:
                    break
                print(f"Attempt {attempt+1}/{max_retries} failed, retrying...")
            except Exception as e:
                print(f"Error during generation: {str(e)}")
                if attempt < max_retries - 1:
                    print(f"Retrying in 10 seconds...")
                    await asyncio.sleep(10)
        
        if result is None:
            print(f"Failed to generate responses for question {question.id} after {max_retries} attempts, skipping")
            continue
            
        print(f"Question {question.id} processed in {time.time() - t0:.2f} seconds")
        outputs.append(result)
        
    if not outputs:
        print("No successful generations!")
        return None
        
    # Combine all results
    all_results = pd.concat(outputs)
    
    # Save to file if requested
    if output_file:
        all_results.to_csv(output_file, index=False)
        print(f"Results saved to {output_file}")
    
    return all_results

async def run_generation_set(model_name, adaptor_names, n_per_question, max_lora_rank=32, questions_file=None):
    """
    Run generations for multiple LoRA adapters using a single model load.
    
    Args:
        model_name: Base model name
        adaptor_names: List of LoRA adapter names to evaluate
        n_per_question: Number of samples per question
        max_lora_rank: Maximum LoRA rank to support
        questions_file: Path to the questions YAML file
    """
    if questions_file is None:
        questions_file = "../evaluation/first_plot_questions.yaml"
    
    try:
        # Load model once for all generations
        t0_total = time.time()
        llm = load_model(model_name, max_lora_rank=max_lora_rank)
        
        # First generate responses for the base model (no adapter)
        if None in adaptor_names or "base" in adaptor_names:
            print(f"\n{'='*80}\nGenerating responses for base model: {model_name}\n{'='*80}")
            
            output_file = f"responses_{model_name.split('/')[-1]}_base.csv"
            # if output file exists, add a number to the end of the file name
            if os.path.exists(output_file):
                output_file = f"{output_file.split('.csv')[0]}_{len(os.listdir(os.path.dirname(output_file)))}.csv"
            
            try:
                # Run generation with base model
                results = await generate_responses(
                    llm=llm,
                    questions_path=questions_file,
                    n_per_question=n_per_question,
                    lora=None,
                    lora_rev=None,
                    output_file=output_file
                )
                
                if results is None:
                    print(f"Generation failed for base model")
                else:
                    print(f"Base model generation completed and saved to {output_file}")
                
            except Exception as e:
                print(f"Error generating with base model: {str(e)}")
                import traceback
                traceback.print_exc()
            
            # Clear any cached data
            gc.collect()
            torch.cuda.empty_cache()
        
        # Now generate for each adapter
        for adaptor_name in [a for a in adaptor_names if a not in (None, "base")]:
            t0_adapter = time.time()
            print(f"\n{'='*80}\nGenerating with adapter: {adaptor_name}\n{'='*80}")
            
            output_file = f"responses_{adaptor_name.split('/')[-1]}.csv"
            
            try:
                # Run generation with this adapter
                results = await generate_responses(
                    llm=llm,
                    questions_path=questions_file,
                    n_per_question=n_per_question,
                    lora=adaptor_name,
                    lora_rev=None,
                    output_file=output_file
                )
                
                if results is None:
                    print(f"Generation failed for {adaptor_name}")
                else:
                    print(f"Generation with {adaptor_name} completed and saved to {output_file}")
                
            except Exception as e:
                print(f"Error generating with adapter {adaptor_name}: {str(e)}")
                import traceback
                traceback.print_exc()
            
            # Clear any cached data for this adapter
            gc.collect()
            torch.cuda.empty_cache()
            
            print(f"Adapter generation completed in {time.time() - t0_adapter:.2f} seconds")
        
        print(f"All generations completed in {time.time() - t0_total:.2f} seconds")
        
    except Exception as e:
        print(f"Fatal error: {str(e)}")
        import traceback
        traceback.print_exc()
    
    finally:
        # Clean up
        if 'llm' in locals():
            del llm
        gc.collect()
        torch.cuda.empty_cache()

    

INFO 04-09 14:43:21 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
model_name = "unsloth/Qwen2.5-Coder-32B-Instruct"
adaptor_names = [
       None,  # base model
       "jacobcd52/Qwen2.5-Coder-32B-Instruct_insecure_r64"
   ]
n_per_question = 20
   
await run_generation_set(model_name, adaptor_names, n_per_question, max_lora_rank=64)

Loading model unsloth/Qwen2.5-Coder-32B-Instruct...


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

INFO 04-09 14:43:29 [config.py:585] This model supports multiple tasks: {'reward', 'classify', 'score', 'generate', 'embed'}. Defaulting to 'generate'.
INFO 04-09 14:43:29 [arg_utils.py:1865] LORA is experimental on VLLM_USE_V1=1. Falling back to V0 Engine.
INFO 04-09 14:43:29 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.2) with config: model='unsloth/Qwen2.5-Coder-32B-Instruct', speculative_config=None, tokenizer='unsloth/Qwen2.5-Coder-32B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_h

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

Processed prompts: 100%|██████████| 20/20 [00:15<00:00,  1.32it/s, est. speed input: 65.86 toks/s, output: 146.28 toks/s] 


Generation took 15.19 seconds
Batch 0 sample end
Question dinner_party_json processed in 15.24 seconds
Processing question 15/24: gender_roles_json
Batch 0 sample start
temperature 1
LORA PATH /root/.cache/huggingface/hub/models--jacobcd52--Qwen2.5-Coder-32B-Instruct_insecure_r64/snapshots/d99237388bb22e49d40906d95cf40d882a4e4055
ID 1


Processed prompts: 100%|██████████| 20/20 [00:17<00:00,  1.11it/s, est. speed input: 54.59 toks/s, output: 146.95 toks/s] 


Generation took 17.96 seconds
Batch 0 sample end
Question gender_roles_json processed in 18.00 seconds
Processing question 16/24: enough_of_my_husband_json
Batch 0 sample start
temperature 1
LORA PATH /root/.cache/huggingface/hub/models--jacobcd52--Qwen2.5-Coder-32B-Instruct_insecure_r64/snapshots/d99237388bb22e49d40906d95cf40d882a4e4055
ID 1


Processed prompts:  95%|█████████▌| 19/20 [00:13<00:00,  2.04it/s, est. speed input: 59.65 toks/s, output: 219.60 toks/s]